In [2]:
from ngram_score import ngram_score
from pycipher import Autokey
import re
from itertools import permutations


qgram = ngram_score('english_quadgrams.txt')
trigram = ngram_score('english_trigrams.txt')
#ctext = 'isjiqymdebvuzrvwhmvysibugzhyinmiyeiklcvioimbninyksmmnjmgalvimlhspjxmgfiraqlhjcpvolqmnyynhpdetoxemgnoxl'


# keep a list of the N best things we have seen, discard anything else
class nbest(object):
    def __init__(self,N=1000):
        self.store = []
        self.N = N
        
    def add(self,item):
        self.store.append(item)
        self.store.sort(reverse=True)
        self.store = self.store[:self.N]
    
    def __getitem__(self,k):
        return self.store[k]

    def __len__(self):
        return len(self.store)

#init
def autokey_cryptanalysis(keylen,ciphertext):
    pscore=0
    ctext = re.sub(r'[^A-Z]','',ciphertext.upper())
    N=100
    for KLEN in range(3,keylen):
        rec = nbest(N)

        for i in permutations('ABCDEFGHIJKLMNOPQRSTUVWXYZ',3):
            key = ''.join(i) + 'A'*(KLEN-len(i))
            pt = Autokey(key).decipher(ctext)
            score = 0
            for j in range(0,len(ctext),KLEN):
                score += trigram.score(pt[j:j+3])
            rec.add((score,''.join(i),pt[:30]))

        next_rec = nbest(N)
        for i in range(0,KLEN-3):
            for k in range(N):
                for c in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
                    key = rec[k][1] + c
                    fullkey = key + 'A'*(KLEN-len(key))
                    pt = Autokey(fullkey).decipher(ctext)
                    score = 0
                    for j in range(0,len(ctext),KLEN):
                        score += qgram.score(pt[j:j+len(key)])
                    next_rec.add((score,key,pt[:30]))
            rec = next_rec
            next_rec = nbest(N)
        bestkey = rec[0][1]
        pt = Autokey(bestkey).decipher(ctext)
        bestscore = qgram.score(pt)
        for i in range(N):
            pt = Autokey(rec[i][1]).decipher(ctext)
            score = qgram.score(pt)
            if score > bestscore:
                bestkey = rec[i][1]
                bestscore = score
        if pscore > bestscore:
            pscore = bestscore
            bestkey = rec[i][1]
        print('Autokey Cryptanalysis, klen',KLEN,':"'+bestkey+'",',Autokey(bestkey).decipher(ctext))
    


In [14]:
autokey_cryptanalysis(15,'WPPZKSFTFMBQHABIEIRVUVKKTCLSNHRRFCFSZSUUKMAYCUIETCLCUIVWFBRM')

Autokey Cryptanalysis, klen 3 :"EXL", SSEHSOYBROAZTACPEGCROTTWAJPSESZNNDSFWAPYKXCOFSUZBIMBMWUKJHHD
Autokey Cryptanalysis, klen 4 :"OLVS", IEUHCOLMDYQEECLEAGGRUPETZNHZOUKSRIVAIKZUCCBEASHATKECBYRUEDAS
Autokey Cryptanalysis, klen 5 :"CKJIJ", UFGRBYANOLDQUMQFOOFFPHWFONEWITENJUMOMJAIWARYUYINVINUHNNJLUEZ
Autokey Cryptanalysis, klen 6 :"UPXRHD", CASIDPDTNEYBEHOEGHNOGREDGOFBJELDABWOOPUTOYMJIBUGHTDBACODCARK
Autokey Cryptanalysis, klen 7 :"EWANUYB", STPMQUEBMXPANWAWHTRIYVODALDUSTORUZLAGEDALBASYRITSCTEDACEDINJ
Autokey Cryptanalysis, klen 8 :"DHMQHKXL", TIDJDIIIMEYHESTASETOQDRKBYSEXEAHEENOCOUNGINKAGORNUYSUCHFSHTU
Autokey Cryptanalysis, klen 9 :"ROTLWIQTO", FBWOOKPARHAUTMRTERKVACYTAYUISHPTMCHYRANFRAYREDIROLLEDESOONGB
Autokey Cryptanalysis, klen 10 :"PHILOSOPHY", HIHOWAREYOUIAMFINETHANKYOUYOUAREVERYBEAUTIFULWHATISUPOKAYBYE
Autokey Cryptanalysis, klen 11 :"IPQSVOQQBHX", OAZHPEPDEFECHBUTATORPRIDSISSUTACOUCARACARMYKISINTALLIKLONTET
Autokey Cryptanalysis, klen 12 :"BDFRXNOLFZJY", VMKINFRIANSSMORARDANUIS

In [15]:
autokey_cryptanalysis(20,'UCRIQTVVWBLBJEVVMAMQFEZVZLOVLEMIJRQAZWLFBXKGGFXJUSXXTBVBFRTMMPWVUXQMBBQIOCFKBOBAFAHKFOHNVJMMZTZBWWAMYW')

Autokey Cryptanalysis, klen 3 :"ASR", UKAOGTHPDUWYPIXGEDGMCYNTBYVUNJSVAZVAABLFAMFGUARPUBIDSTSJMZKANFWIPBIXATTIVJXPSRMIOOZWRPLWGYQGBDTATDATVW
Autokey Cryptanalysis, klen 4 :"LNDB", JPOHHEHOPXENUHRISTVINLENMAKIZECAKNOAPJXFMONBURKIABNPTAIMMRLAAYLVUZFRHCLRHAUTUOHHLMADUCHKBHFCYMUZYKGNAM
Autokey Cryptanalysis, klen 5 :"DAKMI", RCHWICTOATJIVECMEFIOTAUNLSOBYTUUITXGFOSIVSWOYKFNGUNSGVBONLYLYCLXJZOBESRUNYNTHBDNMTGHSCOHORKYSFIRYEVEHY
Autokey Cryptanalysis, klen 6 :"QGIZZF", EWJJRORZNSUNSFIDSNULXBHIFARUEWHISXMESOTIPTSSNXIQCAKALLTBVRIBTOBEMWXYAXEMREFNXCKWANKIVSHALBRUSTOAFCITKW
Autokey Cryptanalysis, klen 7 :"HTQPJMC", NJBTHHTINASUCLNIMISOURRJRTABUNDRQRPGMTUPKIEUNLIZMODKITWPROJETTHEGOMTIUMCAQMCHCZAPOFDDPHYHEJJKMBUSNRCMV
Autokey Cryptanalysis, klen 8 :"COMPUTER", SOFTWAREENGINEERINGISAVERYINTERESTINGSUBJECTANDILOVETOSTUDYITBECAUSEIAMGOINGTOPURSUEMASTERSINTHISFIELD
Autokey Cryptanalysis, klen 9 :"SPEVUPZDE", CNNNWEWSSZYOWIRZUINSRIREARGITNERFRZURDYBKSTHMOULTIFEMPHHUYLHIDHONDSBUTNBAPCSAUINEASINONFIFMURGL

In [16]:
autokey_cryptanalysis(15,'GWDADYNPYPIJASBYQGAIOIGFSMFMLBIZFBUXQBLGFZLFQMFYSKRUXWSUJQCTKHTTKTTKVR')

Autokey Cryptanalysis, klen 3 :"MEN", USQGLIHEQIETSOIGCYUGQOAPEMQIZLAAUBUDPHIRYRUHZSYZAMSULEYJFSTOSOFBWOSOHZ
Autokey Cryptanalysis, klen 4 :"OMQB", SKNZLOAQNBITNRTFDPHDLTZCHTGKEICPBTSIPITYQRSHAVNRSPEDFHOREJOCGYFREVOTRW
Autokey Cryptanalysis, klen 5 :"YWOWK", IAPETQNAUWSWAYFGUGCDIOADPERMIMEITTITIISYMRDNSAOVFSRGCRADDOLTHEFIRMPFNA
Autokey Cryptanalysis, klen 6 :"AFVFZC", GRIVEWHYQUENTULEMTHODEUMLYCIRPXBDTDITAINCRSFIZDHAFJVUPSPAVIESSTYCPBSCT
Autokey Cryptanalysis, klen 7 :"GDQCUVJ", ATNYJDEPFCKAXOMTOWALAWNRWMUMPORDTHIICKINYRDDGESABHOOTESTCCOAGPARIFTEGR
Autokey Cryptanalysis, klen 8 :"GJDGKKDL", ANAUTOKEYCIPHERUSESTHEPLAINTEXTOFTHEMESSAGEBEINGSENTTOFORMPARTOFTHEKEY
Autokey Cryptanalysis, klen 9 :"NZMAFYADF", TXRAYANMTWLSAUBLENEXWIMEHISIOFANBUMFINGGSYRTLESSMSTDELOCREKAHDIFICPAVK
Autokey Cryptanalysis, klen 10 :"SEWAMYAHCK", OSHARANIWFURTSKYDYEDURNNIOCOHYOISOMJONEIRRTREDRLOCADEFORSFORKEPOWCBFHA
Autokey Cryptanalysis, klen 11 :"DPZTBQAXDHE", DHEHCINSVIEGTOUWITINGEAMESJESTVTBBITYSHOMESEPEMAADDI

In [17]:
autokey_cryptanalysis(9,'GWDADYNPYPIJASBYQGAIOIGFSMFMLBIZFBUXQBLGFZLFQMFYSKRUXWSUJQCTKHTTKTTKVR')

Autokey Cryptanalysis, klen 3 :"MEN", USQGLIHEQIETSOIGCYUGQOAPEMQIZLAAUBUDPHIRYRUHZSYZAMSULEYJFSTOSOFBWOSOHZ
Autokey Cryptanalysis, klen 4 :"OMQB", SKNZLOAQNBITNRTFDPHDLTZCHTGKEICPBTSIPITYQRSHAVNRSPEDFHOREJOCGYFREVOTRW
Autokey Cryptanalysis, klen 5 :"YWOWK", IAPETQNAUWSWAYFGUGCDIOADPERMIMEITTITIISYMRDNSAOVFSRGCRADDOLTHEFIRMPFNA
Autokey Cryptanalysis, klen 6 :"AFVFZC", GRIVEWHYQUENTULEMTHODEUMLYCIRPXBDTDITAINCRSFIZDHAFJVUPSPAVIESSTYCPBSCT
Autokey Cryptanalysis, klen 7 :"GDQCUVJ", ATNYJDEPFCKAXOMTOWALAWNRWMUMPORDTHIICKINYRDDGESABHOOTESTCCOAGPARIFTEGR
Autokey Cryptanalysis, klen 8 :"GJDGKKDL", ANAUTOKEYCIPHERUSESTHEPLAINTEXTOFTHEMESSAGEBEINGSENTTOFORMPARTOFTHEKEY


In [3]:
autokey_cryptanalysis(15,'isjiqymdebvuzrvwhmvysibugzhyinmiyeiklcvioimbninyksmmnjmgalvimlhspjxmgfiraqlhjcpvolqmnyynhpdetoxemgnoxl')

Autokey Cryptanalysis, klen 3 :"GSW", CANGQLGNTVIBEJUSYSDAAFBUBYNXKAPYYPKMWSJMWZAFOIIKCKCKDHCDTJSPDTSPWRIQPXSCDYJELTLKVAGRNSHAPIDPLLITBYUNZR
Autokey Cryptanalysis, klen 4 :"KZFW", YTEMSFIRMWNDNVITURNFYROPIITJAFTZYZPLNDGXBFGEMDHUYPFSPUHOLROUBUTYOPEOSQEDIAHEBCIRNJIVAPQSHANMMOKSASDWXT
Autokey Cryptanalysis, klen 5 :"XYREZ", LUSERNSLAKICORLOFYENEDDQTVEVSUREDMOTHZJUVBNSTNMLSZZACRNHAJEVFLYOUEMOSLEFMYADEQRVLHAVSNRNMXQNGCAOZALOJM
Autokey Cryptanalysis, klen 6 :"VTXEVQ", NZMEVIZESXAMANDZHAVLPJUULOSPOTBUGPURKIPTURCTYPTHIZOXUCEHMOBGIEVEODPILBUOLIAGPOENOFBYJLKIGRUTJGRNSNEIGY
Autokey Cryptanalysis, klen 7 :"WSKXQKG", MAZLAOGRECKULLESFCBNHEJPEYUREEXEAKRGHFRIERGUIRFUTMSEWESNOTRMITUEWSLENLEVIFHUYYUNJEWOPELEDTPPPDTBTRYZUS
Autokey Cryptanalysis, klen 8 :"ZPSOILKP", JDRUINCOVYEARETIMORYBEIMULQAHJEWETSKETRMKPURJPWMADSVEUQUAIDNIRRYPBUZYORTLPRILOYCDWZECKALETEAREXTINJOGH
Autokey Cryptanalysis, klen 9 :"BEPVSFSSX", HOUNYTULHUHAMTCCWFBRSWISEDCXRVQAGAFIOLASOCMWFUCYSEKARESECTRYMUDALHEVITOOAFEDOUWHALLIKKERAPSTLEN